In [1]:
!pip install torch torchaudio transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, concatenate_datasets
import soundfile as sf
import torchaudio

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# Load Common Voice dataset
common_voice = load_dataset("mozilla-foundation/common_voice_13_0", "ar", split="train+test+validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 988it [00:00, 4215.84it/s]
Reading metadata...: 7919it [00:00, 28867.06it/s]
Reading metadata...: 16338it [00:00, 48288.04it/s]
Reading metadata...: 28167it [00:00, 48403.47it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10409it [00:00, 106208.03it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10445it [00:00, 70363.04it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 11369it [00:00, 113683.06it/s]
Reading metadata...: 36192it [00:00, 130379.99it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14994it [00:00, 125833.65it/s]


In [6]:
print(len(common_voice))

49021


In [7]:
print(common_voice)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 49021
})


In [8]:
# Removing unwanted features (we only want audio path and its transcription)
common_voice = common_voice.remove_columns(["client_id", "audio", "up_votes", "down_votes", "age", "gender", "accent", "locale", "segment", "variant"])

print(common_voice)

Dataset({
    features: ['path', 'sentence'],
    num_rows: 49021
})


In [9]:
print(common_voice[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'sentence': 'وما أدراك ما يوم الدين'}


In [10]:
fleurs_train = load_dataset("google/fleurs", "ar_eg", split="train")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
fleurs_test = load_dataset("google/fleurs", "ar_eg", split="test")

In [12]:
fleurs_val = load_dataset("google/fleurs", "ar_eg", split="validation")

In [13]:
print(len(fleurs_train))
print(len(fleurs_test))
print(len(fleurs_val))

2104
428
295


In [14]:
print(fleurs_train)

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 2104
})


In [15]:
def update_audio_path_train(data_item):
    parts = data_item['path'].split('/')
    parts.insert(-1, 'train')
    data_item['path'] = '/'.join(parts)
    data_item['sentence'] = data_item['transcription']
    return data_item
def update_audio_path_test(data_item):
    parts = data_item['path'].split('/')
    parts.insert(-1, 'test')
    data_item['path'] = '/'.join(parts)
    data_item['sentence'] = data_item['transcription']
    return data_item
def update_audio_path_val(data_item):
    parts = data_item['path'].split('/')
    parts.insert(-1, 'dev')
    data_item['path'] = '/'.join(parts)
    data_item['sentence'] = data_item['transcription']
    return data_item

In [16]:
# Apply the transformation
fleurs_train = fleurs_train.map(update_audio_path_train)

Map:   0%|          | 0/2104 [00:00<?, ? examples/s]

In [17]:
# Apply the transformation
fleurs_test = fleurs_test.map(update_audio_path_test)

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

In [18]:
# Apply the transformation
fleurs_val = fleurs_val.map(update_audio_path_val)

Map:   0%|          | 0/295 [00:00<?, ? examples/s]

In [19]:
# Removing unwanted features (we only want audio path and its transcription)
fleurs_train = fleurs_train.remove_columns(["id", "num_samples", "audio", "transcription", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [20]:
fleurs_test = fleurs_test.remove_columns(["id", "num_samples", "audio", "transcription", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [21]:
fleurs_val = fleurs_val.remove_columns(["id", "num_samples", "audio", "transcription", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [22]:
print(fleurs_train[0])
print(fleurs_test[0])
print(fleurs_val[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/1f40795a758b41f3dd2aadc7ddb393b3b0158ab7414a341fa1851205b8fd98dd/train/10002899498199807056.wav', 'sentence': 'وعلى الرغم من ذلك فإنها معضلة من الصعب حلها وستستغرق سنين طوال قبل أن نشهد بناء مفاعلات اندماج ذات نفع'}
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/583763dc2530b16401d4dd1552733fe3cde4c859e05cc8f88da57f398747fad4/test/10019390257075855641.wav', 'sentence': 'تشكلت في المحيط الأطلسي اليوم عاشر عاصفة مُسماة لموسم الأعاصير الأطلسية العاصفة شبه الاستوائية جيري'}
{'path': '/root/.cache/huggingface/datasets/downloads/extracted/b78cca77a9b4c6f0130b6ed2d517bcb315ab50a8fa903174dc16cc6a1bf9b197/dev/10013199422658976283.wav', 'sentence': 'قام البرتغاليون بتدميرها وإعادة بنائها تحت اسم كازا برانكا فقط ليهجروها بعد زلزال 1755'}


In [23]:
combined_dataset = concatenate_datasets([common_voice, fleurs_train, fleurs_test, fleurs_val])

In [24]:
print(len(common_voice))
print(len(fleurs_train))
print(len(fleurs_test))
print(len(fleurs_val))
print(len(combined_dataset))

49021
2104
428
295
51848


In [25]:
print(combined_dataset)

Dataset({
    features: ['path', 'sentence'],
    num_rows: 51848
})


In [26]:
print(combined_dataset[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'sentence': 'وما أدراك ما يوم الدين'}


In [27]:
vocabulary = ['ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ء', 'آ', 'أ', 'إ', 'ؤ', 'ئ', 'ة', 'ى', 'ﻻ', 'ﻷ', 'ﻹ', 'ﻵ',' ', '.']

In [28]:
print(len(vocabulary))

42


In [29]:
def process_transcriptions(data_item):
    new_sentence = ''.join([char for char in data_item["sentence"] if char in vocabulary])
    data_item['sentence'] = new_sentence
    return data_item

In [30]:
# Apply the transformation
combined_dataset = combined_dataset.map(process_transcriptions)

Map:   0%|          | 0/51848 [00:00<?, ? examples/s]

In [31]:
print(combined_dataset[0])

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/16dcf9a265646675365f5a6dca5286bafd824e745b8f99fad6d37251d933b4cd/ar_train_0/common_voice_ar_24082672.mp3', 'sentence': 'وما أدراك ما يوم الدين'}


In [32]:
class CustomProcessor:
    def __init__(self, vocab):
        self.vocab = vocab
        self.char_to_index = {char: index for index, char in enumerate(vocab)}

    def text_to_int(self, text):
        return [self.char_to_index[char] for char in text]

    def int_to_text(self, indices):
        return ''.join([self.vocab[index] for index in indices])

In [33]:
object_voc = CustomProcessor(vocabulary)

In [34]:
text = "مرحبا"
encoding = object_voc.text_to_int(text)
decoding = object_voc.int_to_text(encoding)
print(encoding)
print(decoding)

[23, 9, 5, 1, 0]
مرحبا


In [35]:
import torch
from torch.utils.data import Dataset
import torchaudio

In [36]:
class CustomDataset(Dataset):
    def __init__(self, dataset, vocab):
        self.dataset = dataset
        self.processor = CustomProcessor(vocab)
        self.spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load and resample audio
        audio_input, sampling_rate = torchaudio.load(self.dataset[idx]["path"])
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
            audio_input = resampler(audio_input)

        # Convert to spectrogram
        # spectrogram = self.spectrogram_transform(audio_input).squeeze(0)
        spectrogram = self.spectrogram_transform(audio_input)
        # Normalize spectrogram
        spectrogram = (spectrogram - spectrogram.mean()) / spectrogram.std()

        # Process text
        sentence = self.dataset[idx]["sentence"]

        labels = self.processor.text_to_int(sentence)
        labels = torch.tensor(labels)

        return spectrogram, labels, spectrogram.shape[-1]

In [37]:
# Instantiate the dataset
custom_data_set = CustomDataset(combined_dataset, vocabulary)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [38]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [39]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [40]:
import torch
import torch.nn as nn

class CustomSTTModel2(nn.Module):
    def __init__(self, num_classes):
        super(CustomSTTModel2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.pos_encoder = PositionalEncoding(d_model=2048)  # Set d_model to match the input size of GRU1

        self.gru1 = nn.GRU(input_size=2048, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(input_size=256, hidden_size=128, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.5)

        self.fc = nn.Linear(256, 256)
        self.out = nn.Linear(256, num_classes + 1)  # num_classes includes the Arabic charset + 1 for CTC blank

    def forward(self, x):
        # print("cccccccccccccccccccccccccc")
        # x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool(x)
        # print("dddddddddddddddddddddddddddd")
        # x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)
        # print(x.shape)
        # torch.Size([8, 64, 32, 166]) [Batch Size, Channels, Height(frequency), Width(time)]
        # print("eeeeeeeeeeeeeeeeeeeeeeeeeee")
        x = x.permute(0, 3, 1, 2)  # Rearrange dimensions for GRU; as sequence models expect [Sequence Length (Time Steps),  Batch Size, Feature Size (Number of Features per Time Step)]
        # This often involves flattening the non-time dimensions (like channels and frequency bins) into a single feature dimension
        # print(x.shape)
        # torch.Size([8, 166, 64, 32]) [Batch Size, Width, Channels, Height]
        x = torch.flatten(x, start_dim=2)  # Flatten the convolutional features
        # print("fffffffffffffffffffffffffff")
        # print(x.shape)
        # torch.Size([8, 166, 2048]) [Batch Size, Width, Channels * Height]
        # Apply positional encoding
        x = self.pos_encoder(x)

        x, _ = self.gru1(x)
        # print(x.shape)
        # print("gggggggggggggggggggggggggggg")
        x = self.dropout(x)

        x, _ = self.gru2(x)
        # print(x.shape)
        # print("hhhhhhhhhhhhhhhhhhhhhhhhhhhh")
        x = self.dropout(x)


        # x = torch.relu(self.fc(x))
        x = self.fc(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.out(x)
        # print(x.shape)
        # print("iiiiiiiiiiiiiiiiiiiiiiiii")
        return x

In [41]:
# Instantiate the model
custom_model_2 = CustomSTTModel2(len(vocabulary))

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Separate the data items
    spectrograms, labels, input_lengths = zip(*batch)

    spectrograms = [s.permute(2, 0, 1) for s in spectrograms]  # Shape: [time_steps, channels, n_mels] since we need to pad the time_steps and pad_sequence pads the first dimension

    # Pad the spectrograms to have the same time length
    # Note: pad_sequence expects a list of tensors, padding them to match the longest tensor
    spectrograms = pad_sequence(spectrograms, batch_first=True, padding_value=0)

    # Permute back to original dimension order after padding
    spectrograms = spectrograms.permute(0, 2, 3, 1)  # Shape: [batch, channels, n_mels, time_steps]

    # Concatenate all labels lengths
    label_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long, device=device)

    # Concatenate all labels (no need for padding)
    labels = torch.cat(labels)

    # Convert input_lengths into a tensor
    input_lengths = torch.tensor(input_lengths)

    return spectrograms, labels, input_lengths, label_lengths

In [44]:
# Create a DataLoader
batch_size = 8
train_loader = DataLoader(custom_data_set, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [45]:
best_loss = 10000

def train(model, data_loader, criterion, optimizer, epochs, device):
  global best_loss
  model.to(device)
  for epoch in range(epochs):
      model.train()  # Set the model to training mode
      total_loss = 0
      for batch_idx, (spectrograms, labels, input_lengths, label_lengths) in enumerate(train_loader):
          spectrograms, labels = spectrograms.to(device), labels.to(device)
          # Forward pass
          outputs = model(spectrograms) # [batch, output sequence length, classes]
          output_lengths = outputs.shape[1]
          output_lengths = torch.full((outputs.shape[0],), output_lengths, dtype=torch.int64)
          # print(outputs)
          log_probs = torch.nn.functional.log_softmax(outputs, dim=2) # CTC expects log softmax probabilities
          # print(log_probs.shape)

          # The output of the network needs to be in the shape (output sequence length, batch, classes)
          log_probs = log_probs.permute(1, 0, 2)
          # print(log_probs.shape)
          log_probs = log_probs.to(device)
          label_lengths = label_lengths.to(device)
          output_lengths = output_lengths.to(device)
          # print(output_lengths)
          # print(log_probs.shape)
          # print(label_lengths)
          # print(labels.shape)
          # Calculate loss
          loss = criterion(log_probs, labels, output_lengths, label_lengths)
          if torch.isinf(loss): # This is due to a few bad anomalies in the dataset. Removing these anomalies would make this condition irrelevant
            # print("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")
            # print(output_lengths)
            # print(log_probs.shape)
            # print(label_lengths)
            # print(labels.shape)
            # print(outputs)
            # print("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
            # print(log_probs)
            continue

          total_loss += loss.item()

          # Backward pass and optimization
          optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
          optimizer.step()

          # Print progress
          print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}")

      # Compute average loss for the epoch
      avg_loss = total_loss / len(data_loader)

      # Save the model every two epochs
      if epoch % 2 == 0:
          torch.save(model.state_dict(), f'model_state_dict_epoch_{epoch}.pth')

      # Update the best model if current model is better
      if avg_loss < best_loss:
          best_loss = avg_loss
          torch.save(model.state_dict(), 'best_model_state_dict.pth')

In [46]:
from torch import nn
import torch.optim as optim

# Define the Loss Function (Criterion)
# The CTC loss function expects logits as inputs, so ensure your model outputs logits
criterion = nn.CTCLoss(blank=len(vocabulary)).to(device)

In [47]:
# Define the Optimizer
optimizer = optim.Adam(custom_model_2.parameters(), lr=0.001)

In [48]:
# Run the training
epochs = 10  # Define the number of epochs
train(custom_model_2, train_loader, criterion, optimizer, epochs, device)

Epoch [1/10], Batch [1/6481], Loss: 40.2183
Epoch [1/10], Batch [2/6481], Loss: 14.4174
Epoch [1/10], Batch [3/6481], Loss: 11.0734
Epoch [1/10], Batch [4/6481], Loss: 13.2212
Epoch [1/10], Batch [5/6481], Loss: 9.6834
Epoch [1/10], Batch [6/6481], Loss: 5.9907
Epoch [1/10], Batch [7/6481], Loss: 4.7104
Epoch [1/10], Batch [8/6481], Loss: 3.9388
Epoch [1/10], Batch [9/6481], Loss: 4.1392
Epoch [1/10], Batch [10/6481], Loss: 4.0010
Epoch [1/10], Batch [11/6481], Loss: 4.3285
Epoch [1/10], Batch [12/6481], Loss: 3.8766
Epoch [1/10], Batch [13/6481], Loss: 4.0847
Epoch [1/10], Batch [14/6481], Loss: 3.8700
Epoch [1/10], Batch [15/6481], Loss: 3.5918
Epoch [1/10], Batch [16/6481], Loss: 3.7465
Epoch [1/10], Batch [17/6481], Loss: 3.7020
Epoch [1/10], Batch [18/6481], Loss: 5.2534
Epoch [1/10], Batch [19/6481], Loss: 3.8864
Epoch [1/10], Batch [20/6481], Loss: 4.6554
Epoch [1/10], Batch [21/6481], Loss: 3.3923
Epoch [1/10], Batch [22/6481], Loss: 3.7028
Epoch [1/10], Batch [23/6481], Loss: 

KeyboardInterrupt: 

In [79]:
import json

# Path to your JSON file
file_path = 'dataset_info.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# data is now a Python dictionary containing the JSON data
print(data)
first_key = next(iter(data))
first_value = data[first_key]
print(f"First key: {first_key}, First value: {first_value}")

{'description': '', 'citation': '', 'homepage': '', 'license': '', 'features': {'path': {'dtype': 'string', '_type': 'Value'}, 'audio': {'sampling_rate': 16000, '_type': 'Audio'}, 'sentence': {'dtype': 'string', '_type': 'Value'}}, 'builder_name': 'parquet', 'dataset_name': 'mgb2_audios_transcriptions_preprocessed', 'config_name': 'default', 'version': {'version_str': '0.0.0', 'major': 0, 'minor': 0, 'patch': 0}, 'splits': {'train': {'name': 'train', 'num_bytes': 1146316411, 'num_examples': 5842, 'shard_lengths': [3895, 1947], 'dataset_name': 'mgb2_audios_transcriptions_preprocessed'}}, 'download_checksums': {'hf://datasets/BelalElhossany/mgb2_audios_transcriptions_preprocessed@4fdc4791fe8ea154876d9bc1ad1eecd140063287/data/train-00000-of-00003-9d3a91865911d904.parquet': {'num_bytes': 390088784, 'checksum': None}, 'hf://datasets/BelalElhossany/mgb2_audios_transcriptions_preprocessed@4fdc4791fe8ea154876d9bc1ad1eecd140063287/data/train-00001-of-00003-df0c6e7b50c171a8.parquet': {'num_bytes

In [ ]:
# Define a function to load an audio file and transform it to a spectrogram
def transform_audio_to_spectrogram(audio_path, transform):
    waveform, sample_rate = torchaudio.load(audio_path)

    # Convert stereo to mono by averaging the two channels if necessary
    if waveform.shape[0] == 2:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    spectrogram = transform(waveform)
    spectrogram = (spectrogram - spectrogram.mean()) / spectrogram.std()
    return spectrogram

In [ ]:
# Load the trained model
model_path = 'best_model_state_dict.pth'
model = CustomSTTModel2(len(vocabulary))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

CustomSTTModel2(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gru1): GRU(2048, 128, batch_first=True, bidirectional=True)
  (gru2): GRU(256, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=127, bias=True)
)

In [ ]:
# Audio transformation
spectrogram_transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128)

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [ ]:
# Inference function
def infer(audio_path, model, transform, processor):
    spectrogram = transform_audio_to_spectrogram(audio_path, transform)
    spectrogram = spectrogram.unsqueeze(0).to(device)  # Add batch dimension and send to device

    with torch.no_grad():
        outputs = model(spectrogram)
        outputs = torch.nn.functional.log_softmax(outputs, dim=2)
        outputs = outputs.permute(1, 0, 2)  # (T, N, C)

    decoded_preds = torch.argmax(outputs, dim=2)
    decoded_preds = decoded_preds.transpose(0, 1)  # (N, T)
    # Convert output to text using CustomProcessor
    text_output = []
    decoded_preds_list = decoded_preds.flatten().tolist()
    # print(decoded_preds_list)
    pred_text = processor.int_to_text([i for i in decoded_preds_list if i < len(processor.vocab)])

    return pred_text

In [ ]:
object_voc = CustomProcessor(vocabulary)
# Example usage
audio_file = 'r3.wav'
# audio_file = dataset[0]["path"]
predicted_text = infer(audio_file, model, spectrogram_transform, object_voc)
print(predicted_text)

sنuىييsۗللل,ييﺃ!uh
